# **1. Download the dataset and understand the format**

In [0]:
from google.colab import files
uploaded = files.upload()


Saving dataSet.tar.gz to dataSet.tar (5).gz


# **2. Generate the Data Matrix and the Label vector **

In [0]:
import tarfile
tf = tarfile.open("dataSet.tar.gz")
tf.extractall()

EOFError: ignored

In [0]:
# reading the dataset
# pgm -> Portable Gray Map
from matplotlib import pyplot as plt
import numpy as np
path = "orl_faces/"
D = []
label = []
for i in range(1,41):
    for j in range(1,11): # just read 2 images bs
        img = plt.imread(path+"/s"+str(i)+"/"+str(j)+".pgm")
        #plt.imshow(img)
        D.append(np.array(img).flatten())
        label.append(i-1) 
        
D = np.matrix(D) 
label = np.array(label)
labels = label
print(D.shape)

# 3. Split the Dataset into Training and Test sets (10 Points)

In [0]:
#Split the Dataset into Training and Test sets
train_vector = D[1::2,:]
train_label = label[1::2]
test_vector = D[0::2,:]
test_label = label[0::2]


# 5. Classification using **LDA**

In [0]:
#LDA
lists = [[] for i in range(0,40)]
for i in range(0,len(train_label)):
    lists[train_label[i]].append(train_vector[i])


    
mean_vectors = []
for i in range(0,len(lists)):
    mean_vectors.append(np.mean(lists[i],axis = 0))

mean = np.mean(train_vector, axis = 0)


Sb = np.zeros((10304,10304))
for i in range(0,40):
      diff = mean_vectors[i] - mean
      diffTranspose = np.transpose(diff)
      tmp = np.matmul(diffTranspose,diff)
      Sb += 5*tmp
#      Sb = np.add(Sb,(5*(np.matmul(np.transpose(np.subtract(mean_vectors[i],mean)),np.subtract(mean_vectors[i],mean)))))

print(Sb)


[[235278.32  232717.44  233560.64  ... -43174.56  -21703.44  -15646.88 ]
 [232717.44  230668.78  231142.03  ... -40923.52  -20191.43  -14642.46 ]
 [233560.64  231142.03  232523.155 ... -39926.72  -18941.855 -13273.41 ]
 ...
 [-43174.56  -40923.52  -39926.72  ... 360976.48  315665.12  312321.04 ]
 [-21703.44  -20191.43  -18941.855 ... 315665.12  294892.955 289157.81 ]
 [-15646.88  -14642.46  -13273.41  ... 312321.04  289157.81  289627.82 ]]


In [0]:

Z = [[] for i in range(0,40)]
for i in range(0,len(train_vector)):
    label = train_label[i]
    Z[label].append(np.subtract(train_vector[i],mean_vectors[label]))

for i in range(0,40):
    Z[i] = np.array(Z[i]).reshape(5,10304)
    
S = np.zeros((10304,10304))
for i in range(0,40):
    S = np.add(S,(np.matmul(np.transpose(Z[i]),Z[i])))

print(S.shape)    

(10304, 10304)


In [0]:
print(S)

[[ 28496.8  27128.2  27372.4 ...  -3977.2  -3511.6    539. ]
 [ 27128.2  28448.8  27518.6 ...  -2697.2  -2349.2   1781.6]
 [ 27372.4  27518.6  28698.4 ...  -3472.2   -923.2   3416.2]
 ...
 [ -3977.2  -2697.2  -3472.2 ... 140034.   83425.8  64222.2]
 [ -3511.6  -2349.2   -923.2 ...  83425.8  99585.6  82754.4]
 [   539.    1781.6   3416.2 ...  64222.2  82754.4 104148.8]]


In [0]:
from numpy import linalg as LA
from numpy.linalg import pinv 

S_inverse = pinv(S)
v, w = LA.eig(np.matmul(S_inverse,Sb))
print(v,w,w.shape)


[-8.17040892e+10+0.0000000e+00j  8.00159974e+10+0.0000000e+00j
  1.68890728e+01+0.0000000e+00j ...  1.09103088e-07-6.1306872e-08j
  2.09831843e-09+0.0000000e+00j -3.17457339e-08+0.0000000e+00j] [[-4.05782436e-03+0.00000000e+00j -6.43321458e-03+0.00000000e+00j
   4.39331064e-03+0.00000000e+00j ...  1.65415867e-03+7.01627090e-04j
  -2.04722360e-03+0.00000000e+00j  4.62133759e-03+0.00000000e+00j]
 [-2.72195024e-01+0.00000000e+00j -1.25681362e-01+0.00000000e+00j
   2.61710257e-01+0.00000000e+00j ...  9.01901578e-03-9.49526785e-04j
   4.62012532e-03+0.00000000e+00j  4.86930095e-03+0.00000000e+00j]
 [ 8.67706411e-01+0.00000000e+00j  9.59023177e-01+0.00000000e+00j
  -8.76287665e-01+0.00000000e+00j ... -7.77027092e-04-1.77285276e-03j
  -1.33636102e-03+0.00000000e+00j -7.99861995e-03+0.00000000e+00j]
 ...
 [ 1.32010556e-04+0.00000000e+00j -4.76478170e-05+0.00000000e+00j
   5.29341046e-04+0.00000000e+00j ... -3.61113218e-03+7.28939857e-04j
  -3.15890634e-03+0.00000000e+00j -4.72086047e-03+0.0000

In [0]:
# used = [0] * len(v)

# projectionMat = []

# for i in range(0,39):
#   idx = -1
#   for j in range(0,len(v)):
#     if(used[j] != 1):
#       if(idx == -1):
#         idx = j
#       elif(v[idx].real < v[j].real):
#         idx = j
  
#   used[idx] = 1
#   col = w[:,idx]
#   projectionMat.append(np.array([col[k].real for k in range(0,len(col))]).flatten())
  
# projectionMat = np.array(projectionMat)
# projectedData = np.matmul(train_vector,np.transpose(projectionMat))
# projectedTest = np.matmul(test_vector,np.transpose(projectionMat))

# print(projectedTest.shape,projectedData.shape)

idx = v.argsort()[::-1]   
v = v[idx]
w = w[:,idx]
U = w[:39].real
projectedData = np.matmul(train_vector,np.transpose(U))
projectedTest = np.matmul(test_vector,np.transpose(U))


# 6.Tuning

In [0]:
# our knn implementation
# predicted = []
# for i in range(0,len(projectedTest)):
#   nearest = 0
#   for j in range(0,len(projectedData)):
#     dist1 = np.linalg.norm(projectedTest[i]-projectedData[nearest])
#     dist2 = np.linalg.norm(projectedTest[i]-projectedData[j])
#     if dist1 > dist2:
#       nearest = j
# predicted.append(train_label[nearest])
  
# correct = 0
# for i in range(0,len(test_label)):
#   if(test_label[i] == predicted[i]):
#     correct+=1
    
# print(correct/2)  

from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import accuracy_score

for k in range(1,8,2):
    knn = NearestNeighbors(n_neighbors=k, algorithm='auto').fit(projectedData)
    distances, indices = knn.kneighbors(projectedTest)  

   
    correct = 0
    for i in range(0,len(indices)):
      cnt = [0] * 40
      mx = 0
      for j in range(0,len(indices[i])):
          cnt[train_label[indices[i][j]]]+=1
          if cnt[train_label[indices[i][j]]] > cnt[mx] :
              mx = train_label[indices[i][j]]
      
      if mx == test_label[i] :
          correct += 1
               
        
    print("K = " , k , "accuracy = " ,correct/2,"\n")

K =  1 accuracy =  81.5 

K =  3 accuracy =  77.0 

K =  5 accuracy =  74.5 

K =  7 accuracy =  71.0 



# 7.Bonus

In [0]:
#Split the Dataset into Training and Test sets
#Split the Dataset into Training and Test sets
train_vector = []
train_label = []
test_vector = []
test_label = []
for i in range(0,40):
  train_vector.append(D[i*10:i*10+7])
  test_vector.append(D[i*10+7:i*10+10])
  train_label.append(labels[i*10:i*10+7])
  test_label.append(labels[i*10+7:i*10+10])

train_label = np.array(train_label).flatten()
train_vector = np.array(train_vector).reshape(280,10304)
test_label = np.array(test_label).flatten()
test_vector = np.array(test_vector).reshape(120,10304)


In [0]:
#LDA
lists = [[] for i in range(0,40)]
for i in range(0,len(train_label)):
    lists[train_label[i]].append(train_vector[i])


    
mean_vectors = []
for i in range(0,len(lists)):
    mean_vectors.append(np.mean(lists[i],axis = 0))

mean = np.mean(train_vector, axis = 0)


Sb = np.zeros((10304,10304))
for i in range(0,40):
      diff = mean_vectors[i] - mean
      diffTranspose = np.transpose(diff)
      tmp = np.matmul(diffTranspose,diff)
      Sb += 5*tmp
#      Sb = np.add(Sb,(5*(np.matmul(np.transpose(np.subtract(mean_vectors[i],mean)),np.subtract(mean_vectors[i],mean)))))

print(Sb)


In [0]:

Z = [[] for i in range(0,40)]
for i in range(0,len(train_vector)):
    label = train_label[i]
    Z[label].append(np.subtract(train_vector[i],mean_vectors[label]))

for i in range(0,40):
    Z[i] = np.array(Z[i]).reshape(7,10304)
    
S = np.zeros((10304,10304))
for i in range(0,40):
    S = np.add(S,(np.matmul(np.transpose(Z[i]),Z[i])))

print(S.shape)    

In [0]:
from numpy import linalg as LA
from numpy.linalg import pinv 

S_inverse2 = pinv(S)
v2, w2 = LA.eig(np.matmul(S_inverse2,Sb))
print(v2,w2,w2.shape)


In [0]:

idx = v2.argsort()[::-1]   
v2 = v2[idx]
w2 = w2[:,idx]
U = w2[:39].real
projectedData = np.matmul(train_vector,np.transpose(U))
projectedTest = np.matmul(test_vector,np.transpose(U))


In [0]:
# our knn implementation
# predicted = []
# for i in range(0,len(projectedTest)):
#   nearest = 0
#   for j in range(0,len(projectedData)):
#     dist1 = np.linalg.norm(projectedTest[i]-projectedData[nearest])
#     dist2 = np.linalg.norm(projectedTest[i]-projectedData[j])
#     if dist1 > dist2:
#       nearest = j
# predicted.append(train_label[nearest])
  
# correct = 0
# for i in range(0,len(test_label)):
#   if(test_label[i] == predicted[i]):
#     correct+=1
    
# print(correct/2)  

from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import accuracy_score

for k in range(1,8,2):
    knn = NearestNeighbors(n_neighbors=k, algorithm='auto').fit(projectedData)
    distances, indices = knn.kneighbors(projectedTest)  

    correct = 0
    for i in range(0,len(indices)):
      cnt = [0] * 40
      mx = 0
      for j in range(0,len(indices[i])):
          cnt[train_label[indices[i][j]]]+=1
          if cnt[train_label[indices[i][j]]] > cnt[mx] :
              mx = train_label[indices[i][j]]
      
      if mx == test_label[i] :
          correct += 1
               
        
    print("K = " , k , "accuracy = " ,correct * 100 / 120,"\n")